In this analysis, we will use gower distance (https://pypi.org/project/gower/) to build a simple food replacement recommender. In the absence of a particular food, the recommender will tell us which is the food with most similar
nutriotional properties. <br>
Here, we will use the cleaned up dataset generated from the original one.

# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import random as rand

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import DistanceMetric
import gower

# Load data

In [3]:
#food_data = pd.read_excel('data/MyFoodData Nutrition Facts SpreadSheet Release 1.4.xlsx')
food_data = pd.read_excel('data/MyFoodData Nutrition Facts Redux.xlsx')

In [4]:
food_data.head(3)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg),Serving Weight 1 (g),Serving Description 1 (g),Serving Weight 2 (g),Serving Description 2 (g),Serving Weight 3 (g),Serving Description 3 (g),Serving Weight 4 (g),Serving Description 4 (g),Serving Weight 5 (g),Serving Description 5 (g),Serving Weight 6 (g),Serving Description 6 (g),Serving Weight 7 (g),Serving Description 7 (g),Serving Weight 8 (g),Serving Description 8 (g),Serving Weight 9 (g),Serving Description 9 (g),200 Calorie Weight (g)
0,Hearts Of Palm Raw,Vegetables,115,0.20,2.7,25.61,17.16,1.5,0.0,0.046,18,1.69,1806,10,68.0,3.0,8.0,0.0,0.0,0.50,NaN,24.11,69.50,13.0,76.0,-32.101,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,14,3.73,0.644,NaN,0.7,NaN,NaN,NaN,0.050,0.180,0.900,NaN,0.810,NaN,24.0,0.0,24.0,24.0,0.0,NaN,0.0,41.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,5.0,89.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173.913
1,Asparagus,Vegetables,20,0.12,2.2,3.88,1.88,2.1,0.0,0.040,24,2.14,202,14,756.0,38.0,5.6,0.0,0.0,1.13,NaN,1.78,93.22,10.0,40.0,-1.940,0.0,NaN,NaN,0.23,0.65,1.00,0.0,0.0,0.0,NaN,NaN,52.0,2,0.54,0.189,0.158,2.3,NaN,NaN,NaN,0.143,0.141,0.978,0.274,0.091,NaN,52.0,0.0,52.0,52.0,16.0,0.6,0.0,449.0,9.0,0.0,710.0,NaN,NaN,0.0,41.6,0.0,NaN,0.0,50.0,NaN,0.0,0.0,0.0,27.0,84.0,75.0,128.0,104.0,31.0,31.0,75.0,52.0,115.0,91.0,49.0,115.0,508.0,233.0,93.0,71.0,106.0,NaN,0.0,0.0,0.0,134.0,1 cup,12.0,"1 spear, small (5 inch long or less)",16.0,"1 spear, medium (5-1/4 inch to 7 inch long)",20.0,"1 spear, large (7-1/4 inch to 8-1/2 inch)",24.0,"1 spear, extra large (8-3/4 inch to 10 inch long)",3.5,1 spear tip (2 inch long or less),NaN,NaN,NaN,NaN,NaN,NaN,1000.000
2,Asparagus (Cooked),Vegetables,22,0.22,2.4,4.11,1.30,2.0,0.0,0.048,23,0.91,224,14,1006.0,50.0,7.7,0.0,0.0,1.50,NaN,2.11,92.63,29.0,76.0,-2.220,0.0,NaN,NaN,0.08,0.42,0.79,0.0,0.0,0.0,0.0,NaN,54.0,14,0.60,0.165,0.154,6.1,21.9,NaN,NaN,0.162,0.139,1.084,0.225,0.079,NaN,149.0,0.0,149.0,149.0,26.1,0.9,0.0,604.0,0.0,30.0,771.0,NaN,NaN,0.0,50.6,0.0,NaN,0.0,105.0,NaN,0.0,0.0,0.0,29.0,92.0,82.0,140.0,113.0,34.0,34.0,82.0,57.0,125.0,99.0,53.0,125.0,555.0,255.0,101.0,77.0,116.0,NaN,0.0,0.0,0.0,90.0,1/2 cup,60.0,4 spears (1/2 inch base),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Data Preparation

## Remove useless columns

This step will be based on our previous work

In [5]:
serving_cols = food_data.columns[food_data.columns.str.contains('Serving')]
food_data.drop(serving_cols,axis=1,inplace=True)

In [6]:
food_data.drop(['200 Calorie Weight (g)'],axis=1, inplace=True)

## Replace NaNs with 0

In [7]:
food_data.replace(np.nan, 0, inplace=True)

In [8]:
food_data.isna().sum()

Name                   0
Food Group             0
Calories               0
Fat (g)                0
Protein (g)            0
                      ..
Serine (mg)            0
Hydroxyproline (mg)    0
Alcohol (g)            0
Caffeine (mg)          0
Theobromine (mg)       0
Length: 97, dtype: int64

In [9]:
food_data.shape

(420, 97)

In [10]:
food_data.columns

Index(['Name', 'Food Group', 'Calories', 'Fat (g)', 'Protein (g)',
       'Carbohydrate (g)', 'Sugars (g)', 'Fiber (g)', 'Cholesterol (mg)',
       'Saturated Fats (g)', 'Calcium (mg)', 'Iron, Fe (mg)',
       'Potassium, K (mg)', 'Magnesium (mg)', 'Vitamin A, IU (IU)',
       'Vitamin A, RAE (mcg)', 'Vitamin C (mg)', 'Vitamin B-12 (mcg)',
       'Vitamin D (mcg)', 'Vitamin E (Alpha-Tocopherol) (mg)',
       'Added Sugar (g)', 'Net-Carbs (g)', 'Water (g)', 'Omega 3s (mg)',
       'Omega 6s (mg)', 'PRAL score', 'Trans Fatty Acids (g)',
       'Soluble Fiber (g)', 'Insoluble Fiber (g)', 'Sucrose (g)',
       'Glucose (Dextrose) (g)', 'Fructose (g)', 'Lactose (g)', 'Maltose (g)',
       'Galactose (g)', 'Starch (g)', 'Total sugar alcohols (g)',
       'Phosphorus, P (mg)', 'Sodium (mg)', 'Zinc, Zn (mg)', 'Copper, Cu (mg)',
       'Manganese (mg)', 'Selenium, Se (mcg)', 'Fluoride, F (mcg)',
       'Molybdenum (mcg)', 'Chlorine (mg)', 'Thiamin (B1) (mg)',
       'Riboflavin (B2) (mg)', 'Nia

# Food substitute recommender demo

We will use the *gower.topn* function from the gower package. <br>
The function returns the top n matches with respect to a match we pass to the function.

## Select a food

In [11]:
food_data.iloc[1,]

Name                    Asparagus
Food Group             Vegetables
Calories                       20
Fat (g)                      0.12
Protein (g)                   2.2
                          ...    
Serine (mg)                 106.0
Hydroxyproline (mg)           0.0
Alcohol (g)                   0.0
Caffeine (mg)                 0.0
Theobromine (mg)              0.0
Name: 1, Length: 97, dtype: object

## Select all the rest of the foods to pick the best match

In [12]:
food_data.iloc[:,:-1]

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg)
0,Hearts Of Palm Raw,Vegetables,115,0.20,2.70,25.61,17.16,1.5,0.0,0.046,18,1.69,1806,10,68.0,3.0,8.0,0.00,0.0,0.50,0.0,24.11,69.50,13.0,76.0,-32.101,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,140.0,14,3.73,0.644,0.000,0.7,0.0,0.0,0.0,0.050,0.180,0.900,0.000,0.810,0.0,24.0,0.0,24.0,24.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Asparagus,Vegetables,20,0.12,2.20,3.88,1.88,2.1,0.0,0.040,24,2.14,202,14,756.0,38.0,5.6,0.00,0.0,1.13,0.0,1.78,93.22,10.0,40.0,-1.940,0.0,0.0,0.0,0.23,0.65,1.00,0.0,0.00,0.0,0.00,0.0,52.0,2,0.54,0.189,0.158,2.3,0.0,0.0,0.0,0.143,0.141,0.978,0.274,0.091,0.0,52.0,0.0,52.0,52.0,16.0,0.6,0.0,449.0,9.0,0.0,710.0,0.0,0.0,0.0,41.6,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,27.0,84.0,75.0,128.0,104.0,31.0,31.0,75.0,52.0,115.0,91.0,49.0,115.0,508.0,233.0,93.0,71.0,106.0,0.0,0.0,0.0
2,Asparagus (Cooked),Vegetables,22,0.22,2.40,4.11,1.30,2.0,0.0,0.048,23,0.91,224,14,1006.0,50.0,7.7,0.00,0.0,1.50,0.0,2.11,92.63,29.0,76.0,-2.220,0.0,0.0,0.0,0.08,0.42,0.79,0.0,0.00,0.0,0.00,0.0,54.0,14,0.60,0.165,0.154,6.1,21.9,0.0,0.0,0.162,0.139,1.084,0.225,0.079,0.0,149.0,0.0,149.0,149.0,26.1,0.9,0.0,604.0,0.0,30.0,771.0,0.0,0.0,0.0,50.6,0.0,0.0,0.0,105.0,0.0,0.0,0.0,0.0,29.0,92.0,82.0,140.0,113.0,34.0,34.0,82.0,57.0,125.0,99.0,53.0,125.0,555.0,255.0,101.0,77.0,116.0,0.0,0.0,0.0
3,Garden Cress,Vegetables,32,0.70,2.60,5.50,4.40,1.1,0.0,0.023,81,1.30,606,38,6917.0,346.0,69.0,0.00,0.0,0.70,0.0,4.40,89.40,76.0,152.0,-10.753,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,76.0,14,0.23,0.170,0.553,0.9,0.0,0.0,0.0,0.080,0.260,1.000,0.242,0.247,0.0,80.0,0.0,80.0,80.0,19.5,0.2,0.0,4150.0,0.0,0.0,12500.0,0.0,0.0,0.0,541.9,0.0,0.0,239.0,228.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cucumber,Vegetables,15,0.11,0.65,3.63,1.67,0.5,0.0,0.037,16,0.28,147,13,105.0,5.0,2.8,0.00,0.0,0.03,0.0,3.13,95.23,5.0,28.0,-2.445,0.0,0.0,0.0,0.03,0.76,0.87,0.0,0.01,0.0,0.83,0.0,24.0,2,0.20,0.041,0.079,0.3,1.3,0.0,0.0,0.027,0.033,0.098,0.259,0.040,0.0,7.0,0.0,7.0,7.0,6.0,0.1,0.0,45.0,11.0,0.0,23.0,0.0,0.0,0.0,16.4,0.0,0.0,5.0,32.0,0.0,0.0,0.0,0.0,5.0,19.0,21.0,29.0,29.0,6.0,4.0,19.0,11.0,22.0,44.0,10.0,24.0,41.0,196.0,24.0,15.0,20.0,0.0,0.0,0

## Look for the best match for 'asparagus' among the rest of the foods based on all the attributes

In [15]:
gower_dist = gower.gower_topn(food_data.iloc[1:2,2:], food_data.iloc[:,2:], n = len(food_data))

The array returned contains the indeces of the best matches ranked. The first index (i.e., 1) is clearly 'asparagus' as
it's the most similar food compared to itself

In [16]:
gower_dist

{'index': array([  1,  69,  49,   2,  39,  56,  66,  44,  79,  19,  45,  34,  25,
         33,  47,  48,  35,  80,  32,  30,   6,   8,  76,  23,  16,  65,
         12,  46,  18,  50,  78,  41,  73,  26,  51,  55,  24,  74,   4,
         60,  81,  54, 259,  43, 271,  42, 237,  61,  58,  21,  29,  82,
        235, 266, 147,   5, 268, 145, 280, 397,  31, 274, 265, 152,  38,
         28, 264,  75,  57,  63, 234, 231, 209, 248, 276, 418, 262, 260,
        251,  36,  77, 277, 255, 246,  71,  17,  83, 210,  52,  59, 148,
        263, 272, 281, 223, 232,  37,   7, 377, 269, 275, 215, 233, 254,
        236, 376, 273, 282, 258, 247,  53, 267, 261, 241,  11, 214, 240,
        242, 270, 244, 226, 257,  84, 256,  72,  64, 419, 379, 208, 393,
        243, 245, 224, 279, 229, 218, 146, 154, 111,  20, 216,  62, 113,
        225, 239, 253, 404,  67,  22, 417, 284, 324, 278, 149, 358, 406,
        348, 360, 361,   3, 375,  68, 153, 373, 374, 222,  40, 177,  96,
        114, 112,  97, 405, 409,  70,   0,

The second index returns cooked cauliflower, which is the most similar food to asparagus

In [17]:
food_data.iloc[69,:]

Name                   Cooked Cauliflower
Food Group                     Vegetables
Calories                               23
Fat (g)                              0.45
Protein (g)                          1.84
                              ...        
Serine (mg)                          96.0
Hydroxyproline (mg)                   0.0
Alcohol (g)                           0.0
Caffeine (mg)                         0.0
Theobromine (mg)                      0.0
Name: 69, Length: 97, dtype: object

# Food substitute recommender function

## Define a function 

The function will take the food name, and a few other params, to return a set of most similar food ranked by similarity
using Gower Distance dissimilarity

In [139]:
def food_substitute_recommender(food_name: str, food_data: pd.DataFrame, top_matches: int) -> pd.DataFrame:
    """
    This function takes a food and returns a list of ranked foods most similar to it
    food_name: the name of the food we want a substitute for
    data: the dataset containing the matches
    top_matches: how many similar food we want returned
    """
    
    # When food names have parentheses we get an error, we will remove all non alphanumeric chars from
    # both the food name we look up for, and the dataset column we are looking in
    
    food_name = re.sub('[^A-Za-z0-9]+', ' ', food_name).strip()

    # Look for a particular food in the food data and return a pandas df  
    match = food_data.loc[food_data['Name'].str.replace('[^A-Za-z0-9]+', ' ', \
                                                        regex=True).str.strip().str.contains(food_name,case=False)]
    
    # If the resulting df is empty, this means no matches were found
    if match.empty:
        raise Exception("The food was not found. Try another one")

    # Get the index of the found food as an integer
    index_int = match.index.values.astype(int)[0]
    
    # Calculate the most similar foods by matching our target to the rest of the data
    # Notice that we're not using food name and food group as features to match (food_data.iloc[:,2:])
    # Also, notice that the best match will always be the same food, this means that top_matches will include the
    # food we are looking up. We need to skip this and add an element to the list of returned matches (n = top_matches+1)
    gower_dist = gower.gower_topn(food_data.iloc[index_int:index_int+1,2:], food_data.iloc[:,2:], n = top_matches+1)
    
    # We get the most indeces of the most similar foods starting from 1, so that we exclude the food we were looking for
    most_similar_indeces = gower_dist['index'][1:top_matches+1].astype(int)
    
    print(f"""The {top_matches} most similar foods to {food_name} are: """)
    
    return food_data.filter(items = most_similar_indeces, axis=0).reset_index(drop=True)
    

# Test

Let's select a random food from the dataset and look for the best matches

In [140]:
food_data.sample(1)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
225,Cooked Amaranth,Grains and Pasta,102,1.58,3.8,18.69,0.0,2.1,0.0,0.0,47,2.1,135,65,0.0,0.0,0.0,0.0,0.0,0.19,0.0,16.59,75.16,0.0,0.0,2.169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.23,0.0,148.0,6,0.86,0.149,0.854,5.5,0.0,0.0,0.0,0.015,0.022,0.235,0.0,0.113,0.0,22.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
food_substitute_recommender('Cooked Amaranth', food_data, 3)

The 3 most similar foods to Cooked Amaranth are: 


,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
0,Cooked Spelt,Grains and Pasta,127,0.85,5.50,26.44,0.00,3.9,0.0,0.000,10,1.67,143,49,4.0,0.0,0.0,0.00,0.0,0.26,0.0,22.54,66.56,0.0,0.0,3.809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.57,0.0,150.0,5,1.25,0.215,1.091,4.0,0.0,0.0,0.0,0.103,0.030,2.570,0.0,0.080,0.0,13.0,0.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Pudding Coconut,Sweets,111,3.43,2.89,17.37,15.81,0.5,6.0,2.745,101,0.13,136,12,0.0,44.0,0.1,0.33,1.0,0.04,0.0,16.87,75.27,25.0,59.0,2.098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,140.0,178,0.41,0.035,0.000,3.2,0.0,0.0,0.0,0.031,0.157,0.103,0.0,0.035,0.0,4.0,0.0,4.0,4.0,12.5,0.0,44.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,454.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Couscous Plain Cooked,Grains and Pasta,111,0.16,3.77,23.11,0.10,1.4,0.0,0.029,8,0.38,58,8,0.0,0.0,0.0,0.00,0.0,0.13,0.0,21.71,72.23,3.0,60.0,1.123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,22.0,189,0.26,0.041,0.000,27.4,0.0,0.0,0.0,0.063,0.027,0.978,0.0,0.051,0.0,15.0,0.0,15.0,15.0,3.3,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.1,0.0,0.0,22.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
food_data.sample(1)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
403,Raw Peanuts,Beans and Lentils,567,49.24,25.8,16.13,4.72,8.5,0.0,6.279,92,4.58,705,168,0.0,0.0,0.0,0.0,0.0,8.33,0.0,7.63,6.5,3.0,15555.0,6.064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,376.0,18,3.27,1.144,1.934,7.2,0.0,0.0,0.0,0.64,0.135,12.066,1.767,0.348,0.0,240.0,0.0,240.0,240.0,52.5,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24426.0,15558.0,0.0,0.0,0.0,0.0,250.0,883.0,907.0,1672.0,926.0,317.0,331.0,1377.0,1049.0,1082.0,3085.0,652.0,1025.0,3146.0,5390.0,1554.0,1138.0,1271.0,0.0,0.0,0.0,0.0


In [125]:
food_substitute_recommender('Raw Peanuts',food_data,5)

The 5 most similar foods to Raw Peanuts are: 


,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
0,Peanut Butter (Smooth),Beans and Lentils,588,49.54,21.93,23.98,6.50,5.7,0.0,9.519,54,2.16,592,179,0.0,0.0,0.0,0.0,0.0,5.94,0.0,18.28,1.55,12.0,11280.0,5.240,0.0,0.0,0.0,2.39,4.11,0.00,0.0,0.0,0.0,4.57,0.0,335.0,476,2.67,0.571,1.361,40.4,3.1,0.0,0.0,0.107,0.105,13.160,1.044,0.551,0.0,35.0,0.0,35.0,35.0,65.7,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20726.0,11331.0,0.0,12.0,0.0,0.0,228.0,518.0,608.0,1527.0,672.0,262.0,226.0,1187.0,819.0,773.0,2734.0,550.0,905.0,3016.0,5029.0,1419.0,1391.0,1463.0,0.0,0.0,0.0,0.0
1,Fenugreek Seed,Spices and Herbs,323,6.41,23.00,58.35,0.00,24.6,0.0,1.460,176,33.53,770,191,60.0,3.0,3.0,0.0,0.0,0.00,0.0,33.75,8.84,0.0,0.0,-1.336,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,296.0,67,2.50,1.110,1.228,6.3,0.0,0.0,0.0,0.322,0.366,1.640,0.000,0.600,0.0,57.0,0.0,57.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,391.0,898.0,1241.0,1757.0,1684.0,338.0,369.0,1089.0,764.0,1102.0,2465.0,668.0,1020.0,2708.0,3988.0,1306.0,1198.0,1215.0,0.0,0.0,0.0,0.0
2,Salami,Meats,425,37.00,21.70,1.20,1.20,0.0,80.0,13.100,10,1.52,340,22,0.0,0.0,0.0,2.8,0.0,0.00,0.0,1.20,34.60,0.0,0.0,11.223,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,229.0,1890,4.20,0.160,0.070,25.4,41.2,0.0,0.0,0.930,0.330,5.600,1.060,0.550,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18200.0,3600.0,0.0,0.0,0.0,0.0,253.0,1012.0,1084.0,1625.0,1878.0,470.0,289.0,940.0,686.0,1120.0,1373.0,614.0,1336.0,2095.0,3829.0,1553.0,1336.0,903.0,0.0,0.0,0.0,0.0
3,Ground Mustard Seed,Spices and Herbs,508,36.24,26.08,28.09,6.79,12.2,0.0,1.989,266,9.21,738,370,31.0,2.0,7.1,0.0,0.0,5.07,0.0,15.89,5.27,3792.0,6171.0,14.654,0.0,0.0,0.0,3.69,2.88,0.02,0.0,0.0,0.2,0.00,0.0,828.0,13,6.08,0.645,2.448,208.1,0.0,0.0,0.0,0.805,0.261,4.733,0.810,0.397,0.0,162.0,0.0,162.0,162.0,122.7,1.9,0.0,18.0,0.0,0.0,568.0,0.0,0.0,0.0,5.4,0.0,0.0,22518.0,10088.0,0.0,0.0,0.0,0.0,256.0,838.0,1183.0,2080.0,1844.0,483.0,680.0,1173.0,809.0,1509.0,1933.0,878.0,1173.0,2435.0,5265.0,1587.0,2810.0,759.0,0.0,0.0,0.0,0.0
4,Peanut Bar,Sweets,522,33.70,15.50,47.40,42.22,4.1,0.0,4.678,78,0.97,407,110,0.0,0.0,0.0,0.0,0.0,4.10,0.0,43.30,1.60,2.0,10647.0,6.496,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,308.0,156,3.94,0.774,1.243,4.7,0.0,0.0,0.0,0.100,0.140,7.922,0.830,0.153,0.

In [136]:
food_data.sample(1)

,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
216,Quinoa Cooked,Grains and Pasta,120,1.92,4.4,21.3,0.87,2.8,0.0,0.231,17,1.49,172,64,5.0,0.0,0.0,0.0,0.0,0.63,0.0,18.5,71.61,15.0,974.0,2.247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.63,0.0,152.0,7,1.09,0.192,0.631,2.8,0.0,0.0,0.0,0.107,0.11,0.412,0.0,0.123,0.0,42.0,0.0,42.0,42.0,23.0,0.0,0.0,3.0,0.0,0.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,528.0,1078.0,0.0,0.0,0.0,15.0,52.0,131.0,157.0,261.0,239.0,96.0,63.0,185.0,83.0,185.0,340.0,127.0,183.0,353.0,580.0,216.0,240.0,176.0,0.0,0.0,0.0,0.0


In [137]:
food_substitute_recommender('Quinoa Cooked',food_data,10)

The 10 most similar foods to Quinoa Cooked are: 


,Name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",Magnesium (mg),"Vitamin A, IU (IU)","Vitamin A, RAE (mcg)",Vitamin C (mg),Vitamin B-12 (mcg),Vitamin D (mcg),Vitamin E (Alpha-Tocopherol) (mg),Added Sugar (g),Net-Carbs (g),Water (g),Omega 3s (mg),Omega 6s (mg),PRAL score,Trans Fatty Acids (g),Soluble Fiber (g),Insoluble Fiber (g),Sucrose (g),Glucose (Dextrose) (g),Fructose (g),Lactose (g),Maltose (g),Galactose (g),Starch (g),Total sugar alcohols (g),"Phosphorus, P (mg)",Sodium (mg),"Zinc, Zn (mg)","Copper, Cu (mg)",Manganese (mg),"Selenium, Se (mcg)","Fluoride, F (mcg)",Molybdenum (mcg),Chlorine (mg),Thiamin (B1) (mg),Riboflavin (B2) (mg),Niacin (B3) (mg),Pantothenic acid (B5) (mg),Vitamin B6 (mg),Biotin (B7) (mcg),Folate (B9) (mcg),Folic acid (mcg),Food Folate (mcg),Folate DFE (mcg),Choline (mg),Betaine (mg),Retinol (mcg),"Carotene, beta (mcg)","Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),Vitamin D2 (ergocalciferol) (mcg),Vitamin D3 (cholecalciferol) (mcg),Vitamin D (IU) (IU),Vitamin K (mcg),Dihydrophylloquinone (mcg),Menaquinone-4 (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)","18:3 n-3 c,c,c (ALA) (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Tryptophan (mg),Threonine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Cystine (mg),Phenylalanine (mg),Tyrosine (mg),Valine (mg),Arginine (mg),Histidine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg),Alcohol (g),Caffeine (mg),Theobromine (mg)
0,Cooked Wild Rice,Grains and Pasta,101,0.34,3.99,21.34,0.73,1.8,0.0,0.049,3,0.60,101,32,3.0,0.0,0.0,0.0,0.0,0.24,0.0,19.54,73.93,95.0,119.0,1.977,0.0,0.0,0.0,0.33,0.20,0.20,0.0,0.00,0.0,0.00,0.0,82.0,3,1.34,0.121,0.282,0.8,0.0,0.0,0.0,0.052,0.087,1.287,0.154,0.135,0.0,26.0,0.0,26.0,26.0,10.2,0.0,0.0,2.0,0.0,0.0,64.0,0.0,0.0,0.0,0.5,0.0,0.0,50.0,213.0,0.0,0.0,0.0,0.0,49.0,127.0,167.0,276.0,170.0,119.0,47.0,195.0,169.0,232.0,308.0,104.0,223.0,384.0,695.0,182.0,140.0,211.0,0.0,0.0,0.0,0.0
1,Cooked Millet,Grains and Pasta,119,1.00,3.51,23.67,0.13,1.3,0.0,0.172,3,0.63,62,44,3.0,0.0,0.0,0.0,0.0,0.02,0.0,22.37,71.41,28.0,480.0,2.916,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,100.0,2,0.91,0.161,0.272,0.9,0.0,0.0,0.0,0.106,0.082,1.330,0.171,0.108,0.0,19.0,0.0,19.0,19.0,11.2,0.0,0.0,2.0,0.0,0.0,70.0,0.0,0.0,0.0,0.3,0.0,0.0,184.0,508.0,0.0,0.0,0.0,0.0,38.0,113.0,148.0,446.0,67.0,70.0,67.0,185.0,108.0,184.0,122.0,75.0,314.0,231.0,763.0,92.0,279.0,205.0,0.0,0.0,0.0,0.0
2,Baked Beans,Beans and Lentils,155,5.15,5.54,21.63,0.00,5.5,5.0,1.948,61,1.99,358,43,0.0,0.0,1.1,0.0,0.0,0.00,0.0,16.13,65.17,147.0,593.0,-2.730,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,109.0,422,0.73,0.159,0.255,5.7,0.0,0.0,0.0,0.136,0.049,0.408,0.155,0.090,0.0,48.0,0.0,48.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2133.0,740.0,0.0,0.0,0.0,0.0,67.0,228.0,242.0,428.0,379.0,86.0,62.0,287.0,155.0,282.0,356.0,153.0,236.0,637.0,841.0,232.0,239.0,288.0,0.0,0.0,0.0,0.0
3,Cooked Yellow Sweet Corn,Vegetables,96,1.50,3.41,20.98,4.54,2.4,0.0,0.197,3,0.45,218,26,263.0,13.0,5.5,0.0,0.0,0.09,0.0,18.58,73.41,18.0,586.0,-0.803,0.0,0.0,0.0,2.74,0.84,0.79,0.0,0.17,0.0,7.17,0.0,77.0,1,0.62,0.049,0.167,0.2,0.0,0.0,0.0,0.093,0.057,1.683,0.792,0.139,0.0,23.0,0.0,23.0,23.0,29.1,0.0,0.0,66.0,23.0,0.0,906.0,0.0,0.0,0.0,0.4,0.0,0.0,374.0,603.0,0.0,0.0,0.0,0.0,23.0,133.0,133.0,358.0,141.0,69.0,27.0,155.0,126.0,191.0,135.0,91.0,304.0,252.0,655.0,131.0,301.0,158.0,0.0,0.0,0.0,0.0
4,Yellow Sweet Corn,Vegetables,86,1.35,3.27,18.70,6.26,2.0,0.0,0.325,2,0.52,270,37,187.0,9.0,6.8,0.0,0.0,0.07,0.0,16.70,76.05,14.0,472.0,-1.801,0.0,0.0,0.0,0.89,3.43,1.94,0.0,0.00,0.0,5.70,0.0,89.0,15,0.46,0.054,0.163,0.6,0.0,0.0,0.0,0.155,0.055,1.770,0.717,0.093,0.0,42.0,0.0,42.0,42.0,23.0,0.0,0.0,47.0,16.0,0.0,644.0,0.0,0.0,0.0,0.3,0